# Confusion Matrix
Computes the confusion matrix for a classification problem

In [ ]:
!pip3 install tensorflow==2.4.0 scikit-learn==0.24.1 wget==3.2

In [ ]:
import wget
wget.download(
    'https://raw.githubusercontent.com/'
    'elyra-ai/component-library/master/claimed_utils.py'
)

In [ ]:
import os
import tensorflow as tf
from tensorflow import keras
from claimed_utils import unzip
import os.path
import glob
import numpy as np
import pandas as pd
from sklearn import metrics

In [ ]:
# @dependency codait_utils.ipynb
# @dependency model.zip
# @dependency data.zip
# @param model zip file name
# @param data zip file name
# @returns confusion matrix inline in jupyter
# (next version will return congusion matrix as csv)

In [ ]:
model_zip = os.environ.get('model_zip', 'model.zip')
data_zip = os.environ.get('data_zip', 'data.zip')

In [ ]:
unzip('.', model_zip)
unzip('.', data_zip)

In [ ]:
model = keras.models.load_model('model')

In [ ]:
folder = glob.glob("data/*")
num_classes = len(folder)

batch_size = 32
img_height = 400
img_width = 400
input_shape = (img_width, img_height)


train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    'data',
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    'data',
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

train_ds = train_ds.map(lambda x, y: (x, tf.one_hot(y, depth=num_classes)))
val_ds = val_ds.map(lambda x, y: (x, tf.one_hot(y, depth=num_classes)))

In [ ]:
classes = model.predict(
    val_ds,
    batch_size=batch_size
)

In [ ]:
len(classes)

In [ ]:
y_test = None
for features, labels in val_ds:
    if y_test is not None:
        y_test = np.vstack((y_test, labels.numpy()))
    else:
        y_test = labels.numpy()

In [ ]:
matrix = metrics.confusion_matrix(
    y_test.argmax(axis=1),
    classes.argmax(axis=1)
)

In [ ]:
pd.DataFrame(matrix)